<img width="8%" alt="LinkedIn.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/LinkedIn.png" style="border-radius: 15%">

# LinkedIn - Get polls from post
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=LinkedIn+-+Get+polls+from+post:+Error+short+description">Bug report</a>

**Tags:** #linkedin #post #polls #naas_drivers #content #analytics #image #html #plotly

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Last update:** 2023-11-13 (Created: 2022-01-27)

**Description:** This notebook allows users to get poll results from their LinkedIn posts.


<div class="alert alert-info" role="info" style="margin: 10px">
<b>Disclaimer:</b><br>
This code is in no way affiliated with, authorized, maintained, sponsored or endorsed by Linkedin or any of its affiliates or subsidiaries. It uses an independent and unofficial API. Use at your own risk.

This project violates Linkedin's User Agreement Section 8.2, and because of this, Linkedin may (and will) temporarily or permanently ban your account. We are not responsible for your account being banned.
<br>
</div>

## Input

### Import libraries

In [ ]:
from naas_drivers import linkedin
import plotly.express as px
import naas

### Setup variables
**Mandatory**

[Learn how to get your cookies on LinkedIn](https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75)
- `li_at`: Cookie used to authenticate Members and API clients.
- `JSESSIONID`: Cookie used for Cross Site Request Forgery (CSRF) protection and URL signature validation.
- `linkedin_url`: LinkedIn post URL

In [ ]:
# Mandatory
li_at = naas.secret.get("LINKEDIN_LI_AT") or "YOUR_LINKEDIN_LI_AT" #example: AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2
JSESSIONID = naas.secret.get("LINKEDIN_JSESSIONID") or "YOUR_LINKEDIN_JSESSIONID" #example: ajax:8379907400220387585
linkedin_url = ""

## Model

### Get poll results

In [ ]:
df = linkedin.connect(JSESSIONID, JSESSIONID).post.get_polls(linkedin_url)
print("📝 Nb of poll results", len(df))
df.head(5)

## Output

### Save your result in csv

In [ ]:
df.to_csv("POLL.csv", index=False)
print("💾 Poll results saved in csv")

### Create, export and share your graph

In [ ]:
def create_polls_graph(df):
    poll_id = df.POLL_ID.unique()[0]
    title = df.POLL_QUESTION.unique()[0]

    # Create dataframe
    df = df.groupby(["POLL_RESULT"], as_index=False).agg({"PROFILE_ID": "count"})
    df["VALUE"] = df["PROFILE_ID"] / df["PROFILE_ID"].sum() * 100
    df["VALUE_D"] = df["VALUE"].map("{:.0f}%".format)

    # Count voters
    voters = df.PROFILE_ID.sum()

    # Create fig
    fig = px.bar(
        df,
        y="POLL_RESULT",
        x="PROFILE_ID",
        orientation="h",
        title=f"{title}<br><span style='font-size: 13px;'>Total amount of votes: {voters}</span>",
        text="VALUE_D",
        labels={
            "POLL_RESULT": "Options",
            "PROFILE_ID": "Nb of votes",
            "VALUE_D": "% of votes",
        },
    )
    fig.update_traces(marker_color="black")
    fig.update_layout(
        plot_bgcolor="#ffffff",
        width=600,
        height=400,
        font=dict(family="Arial", size=14, color="black"),
        paper_bgcolor="white",
        xaxis_title=None,
        xaxis_showticklabels=False,
        yaxis_title=None,
        margin_pad=10,
    )
    fig.write_html(f"{poll_id}.html")
    fig.show()
    asset = naas.asset.add(f"{poll_id}.html")
    return asset


create_polls_graph(df)